In [ ]:
# Some of the functions below require an older version of torchtext than the default one Kaggle gives you.
# Make sure that Internet is turned on.
!pip install torchtext==0.10.0
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.


In [ ]:
# This block handles some basic setup and data loading.  
# You shouldn't need to edit this, but if you want to 
# import other standard python packages, that is fine.

# imports
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

from torchtext.legacy import data
from torchtext.legacy import datasets

# download and load the data
text_field = data.Field()
datasets = datasets.WikiText2.splits(root='.', text_field=text_field)

train_dataset, validation_dataset, test_dataset = datasets

text_field.build_vocab(train_dataset, validation_dataset, test_dataset)
vocab = text_field.vocab
vocab_size = len(vocab)

train_text = train_dataset.examples[0].text # a list of tokens (strings)
validation_text = validation_dataset.examples[0].text

print(validation_text[:30])

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:01<00:00, 2.48MB/s]


extracting
['<eos>', '=', 'Homarus', 'gammarus', '=', '<eos>', '<eos>', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean']


In [ ]:
text_field.vocab.freqs['.']

90077

In [ ]:
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

We've implemented a unigram model here as a demonstration.

In [ ]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in vocab.itos]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.
        
        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter 
            # as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:', 
      unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities, 
    it is still good to have a separate more efficient method that only computes 
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab.stoi[validation_text[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob))

    perplexity = math.exp(-np.mean(log_probabilities))
    your_perplexity = model.perplexity(validation_text[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)

unigram validation perplexity: 965.0860734119312


To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [ ]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(vocab.itos, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

<eos> <eos> @-@ Eva , include Sporting important doctors research 26 = protected the a of , Mustangs . four board economy


In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this recent paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following function will be used to generate the required output files.

In [ ]:
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes_short.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab_short.txt

def save_truncated_distribution(model, filename, short=True):
    """Generate a file of truncated distributions.
    
    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    vocab_name = 'eval_output_vocab'
    prefixes_name = 'eval_prefixes'

    if short: 
      vocab_name += '_short'
      prefixes_name += '_short'

    with open('{}.txt'.format(vocab_name), 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = [vocab.stoi[s] for s in eval_vocab]

    all_selected_probabilities = []
    with open('{}.txt'.format(prefixes_name), 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.tqdm_notebook(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

--2023-02-09 00:41:23--  https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519055 (507K) [text/plain]
Saving to: ‘eval_prefixes.txt’

eval_prefixes.txt   100%[===================>] 506.89K  --.-KB/s    in 0.004s  

2023-02-09 00:41:23 (141 MB/s) - ‘eval_prefixes.txt’ saved [519055/519055]

--2023-02-09 00:41:23--  https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12497 (12K) [text/plain]
Saving to: ‘eval_output_vocab.txt’

eval_output_v

In [ ]:
save_truncated_distribution(unigram_demonstration_model, 
                            'unigram_demonstration_predictions.npy')

<ipython-input-6-7710a2f3169b>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm.tqdm_notebook(lines, leave=False):


  0%|          | 0/1000 [00:00<?, ?it/s]

saved unigram_demonstration_predictions.npy


### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{C(w_1,w_2)+\alpha}{C(w_1)+N\alpha}$$

where $N$ is the vocab size and $C$ is the count for the given bigram.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this project.

A properly implemented bi-gram model should get a perplexity below 510 on the validation set.

**Note**: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass. 
Also, we suggest pre-computing and caching the counts $C$ when you initialize `NGramModel` for efficiency. 

In [ ]:
class NGramModel:
    def __init__(self, train_text, n=2, alpha=3e-3):
        # get counts and perform any other setup
        self.n = n
        self.smoothing = alpha

        # YOUR CODE HERE
        n_grams = [' '.join(train_text[i:i+n]) for i in range(len(train_text)-n)]
        self.counts_n = Counter(n_grams)
        self.total_count_n = len(n_grams)
        
        n_1_grams = [' '.join(train_text[i:i+n-1]) for i in range(len(train_text)-n)]
        self.counts_n_1 = Counter(n_1_grams)
        self.total_count_n_1 = len(n_grams)
        
        self.N =len(vocab)


    def n_gram_probability(self, n_gram):
        """Return the probability of the last word in an n-gram.
        
        n_gram -- a list of string tokens
        returns the conditional probability of the last token given the rest.
        """
        assert len(n_gram) == self.n

        # YOUR CODE HERE
        prefix = ' '.join(n_gram[:-1])
        n_gram = ' '.join(n_gram)
        if prefix not in self.counts_n_1:
            return 1/self.N
        return (self.counts_n[n_gram] + self.smoothing) / (self.counts_n_1[prefix] + self.N*self.smoothing)



    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""

        # YOUR CODE HERE
        # use your function n_gram_probability
        # vocab.itos contains a list of words to return probabilities for
        if len(text_prefix) < self.n-1:
            return [1/self.N]*len(vocab.itos)
        ret = []
        if self.n == 1:
            text_prefix = []
        else:
            text_prefix = text_prefix[-(self.n-1):]
        for w in vocab.itos:
            n_gram = text_prefix + [w]
            ret.append(self.n_gram_probability(n_gram))
        return ret



    def perplexity(self, full_text):
        """ full_text is a list of string tokens
        return perplexity as a float """

        # YOUR CODE HERE
        # use your function n_gram_probability
        # This method should differ a bit from the example unigram model because 
        # the first n-1 words of full_text must be handled as a special case.
        log_probabilities = []
        for i in range(1, len(full_text)+1):
            if i < self.n:
                log_probabilities.append(math.log(1/self.N, 2))
            else:
                n_gram = full_text[i-self.n:i]
                log_probabilities.append(math.log(self.n_gram_probability(n_gram), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_model = NGramModel(train_text, 1)
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text)) # this should be the almost the same as our unigram model perplexity above

bigram_model = NGramModel(train_text, n=2)
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text)) # this won't do very well...

save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes

unigram validation perplexity: 965.0913606943268
bigram validation perplexity: 504.4293802356155
trigram validation perplexity: 2965.6640566230676


<ipython-input-6-7710a2f3169b>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm.tqdm_notebook(lines, leave=False):


  0%|          | 0/1000 [00:00<?, ?it/s]

saved bigram_predictions.npy


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.  (We will use this to help us calibrate our scoring on the test set.)

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Bigram validation perplexity: 504.4293802356155

We can also generate samples from the model to get an idea of how it is doing.

In [ ]:
print(generate_text(bigram_model))

We now free up some RAM, **it is important to run the cell below, otherwise you will likely run out of RAM in the Colab runtime.**

In [ ]:
# Free up some RAM. 
del bigram_model
del trigram_model

This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|w_{i-n+1}^{i-1}\right)=\frac{max\left\{C(w_{i-n+1}^i)-\delta,0\right\}}{\sum_{w_i} C(w_{i-n+1}^i)} + \alpha(w_{i-n+1}^{i-1}) P(w_i|w_{i-n+2}^{i-1})$$

$$\alpha\left(w_{i-n+1}^{i-1}\right)=\frac{\delta N_{1+}(w_{i-n+1}^{i-1})}{{\sum_{w_i} C(w_{i-n+1}^i)}}$$
where $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation).  If $\sum_{w_i} C(w_{i-n+1}^i)=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $\delta$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity below 275.

In [ ]:
class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text, lower_order_model, n=2, delta=0.9):
        super().__init__(train_text, n=n)
        self.lower_order_model = lower_order_model
        self.discount = delta

        # YOUR CODE HERE
        self.n_plus = {}
        for i in range(len(train_text)-n):
            n_gram = train_text[i:i+n]
            prefix, last = ' '.join(n_gram[:-1]), ' '.join(n_gram[-1])
            if prefix not in self.n_plus:
                self.n_plus[prefix] = {}
            if last not in self.n_plus[prefix]:
                self.n_plus[prefix][last] = 1



    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n

        # YOUR CODE HERE
        # back off to the lower_order model with n'=n-1 using its n_gram_probability function
        prefix = ' '.join(n_gram[:-1])
        if prefix not in self.n_plus:
            return self.lower_order_model.n_gram_probability(n_gram[1:])
        first = max(self.counts_n[' '.join(n_gram)]-self.discount, 0) / self.counts_n_1[prefix]
        second = self.discount * len(self.n_plus[prefix]) / self.counts_n_1[prefix] * self.lower_order_model.n_gram_probability(n_gram[1:])
        return first + second
        


bigram_backoff_model = DiscountBackoffModel(train_text, unigram_model, 2)
trigram_backoff_model = DiscountBackoffModel(train_text, bigram_backoff_model, 3)
check_validity(trigram_backoff_model)
print('trigram backoff validation perplexity:', trigram_backoff_model.perplexity(validation_text))

trigram backoff validation perplexity: 271.1216015764706


Free up RAM. 

In [ ]:
# Release models we don't need any more. 
del unigram_model
del bigram_backoff_model
del trigram_backoff_model

Fill in your trigram backoff perplexity here.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Trigram backoff validation perplexity: 271.1216015764706


Free up RAM. 

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity below 226.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer (ie. 4 layers total). 
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with `nn.Linear`, then using `F.embedding` with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 after first 2 hidden layers. **Note: You will likely find a performance drop if you add a nonlinear activation function after the dimension reduction layer.**
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
* do early stopping based on validation set perplexity (see Project 0)


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, though.

In [ ]:
def ids(tokens):
    return [vocab.stoi[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

class NeuralNgramDataset(torch.utils.data.Dataset):
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab.stoi['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y

class NeuralNGramNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self, n):
        super().__init__()
        self.n = n
        self.p = 0.2

        # YOUR CODE HERE
        self.embedding_weight = nn.Parameter(torch.rand(vocab_size, 128) / np.sqrt(128))
        self.layers = nn.Sequential(
            nn.Linear((n-1)*128, 1024),
            nn.ReLU(),
            nn.Dropout(self.p),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(self.p),
            nn.Linear(1024, 1024),
            nn.ReLU()
        )
        self.output = nn.Linear(1024, 128)



    def forward(self, x):
        # x is a tensor of inputs with shape (batch, n-1)
        # this function returns a tensor of log probabilities with shape (batch, vocab_size)

        # YOUR CODE HERE
        embedding = F.embedding(x, self.embedding_weight).view(x.shape[0], (self.n-1)*128)
        out = self.layers(embedding)
        out =  self.output(out)
        return F.linear(out, self.embedding_weight)



class NeuralNGramModel:
    # a class that wraps NeuralNGramNetwork to handle training and evaluation
    # it's ok if this doesn't work for unigram modeling
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).cuda()

    def train(self):
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=True)
        # iterating over train_loader with a for loop will return a 2-tuple of batched tensors
        # the first tensor will be previous token ids with size (batch, n-1),
        # and the second will be the current token id with size (batch, )
        # you will need to move these tensors to GPU, e.g. by using the Tensor.cuda() function.

        # this will take some time to run; use tqdm.tqdm_notebook to get a progress bar 
        # (see Project 1a for example)

        # YOUR CODE HERE
        self.network.train()
        optim = torch.optim.Adam(self.network.parameters())
        for epoch in range(10):
            for prefix, last in tqdm.notebook.tqdm(train_loader, leave=False):
                optim.zero_grad()
                prefix = prefix.cuda()
                last = last.cuda()
                pred = self.network(prefix)
                loss = F.cross_entropy(pred, last)
                loss.backward()
                optim.step()
            print('Epoch:', epoch, loss.item())
                



    def next_word_probabilities(self, text_prefix):
        # YOUR CODE HERE
        # don't forget self.network.eval()
        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # if your `perplexity` function below is based on a NeuralNgramDataset DataLoader, you will need to use the same strategy for prefixes with less than n-1 tokens to pass the validity check
        #   the data loader appends extra "<eos>" (end of sentence) tokens to the start of the input so there are always enough to run the network
        if len(text_prefix) < self.n-1:
            text_prefix = ['<eos>']*(self.n-1 - len(text_prefix)) + text_prefix
        prefix = torch.tensor(ids(text_prefix[-(self.n-1):])).cuda().unsqueeze(0)
        self.network.eval()
        with torch.no_grad():
            pred = self.network(prefix)
            prob = F.softmax(pred)
        self.network.train()
        return prob.squeeze(0).cpu().numpy()
        


    def perplexity(self, text):
        # you may want to use a DataLoader here with a NeuralNgramDataset
        # don't forget self.network.eval()

        # YOUR CODE HERE
        sum_log_prob = 0
        dataset = NeuralNgramDataset(ids(text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=1024, shuffle=False)
        num_inputs = len(dataset)
        self.network.eval()
        with torch.no_grad():
            for prefix, last in tqdm.notebook.tqdm(train_loader, leave=False):
                prefix = prefix.cuda()
                last = last.cuda()
                pred = self.network(prefix)
                prob = F.softmax(pred, dim=1)
                prob = prob[torch.arange(prob.shape[0]), last]
                sum_log_prob += torch.sum(torch.log2(prob)).item()
        return 2 ** -(sum_log_prob / num_inputs)



neural_trigram_model = NeuralNGramModel(3)
check_validity(neural_trigram_model)
print("Validity check done")
neural_trigram_model.train()
print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy', short=False)

<ipython-input-12-d6fa2c0cfe4c>:109: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(pred)


  0%|          | 0/1 [00:00<?, ?it/s]

Validity check done


  0%|          | 0/8159 [00:00<?, ?it/s]

Epoch: 0 5.247364521026611


...

  0%|          | 0/213 [00:00<?, ?it/s]

neural trigram validation perplexity: 212.5300276874555


<ipython-input-6-7710a2f3169b>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm.tqdm_notebook(lines, leave=False):


  0%|          | 0/5000 [00:00<?, ?it/s]

saved neural_trigram_predictions.npy


Fill in your neural trigram perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Neural trigram validation perplexity: 212.5300276874555

Free up RAM.

In [ ]:
# Delete model we don't need. 
del neural_trigram_model

### LSTM Model

For this stage of the project, you will implement an LSTM language model.

For recurrent language modeling, the data batching strategy is a bit different from what is used in some other tasks.  Sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch.  We'll use the `torchtext` library to manage this batching for you.  To properly deal with this input format, you should save the last state of the LSTM from a batch to feed in as the first state of the next batch.  When you save state across different batches, you should call `.detach()` on the state tensors before the next batch to tell PyTorch not to backpropagate gradients through the state into the batch you have already finished (which will cause a runtime error).

We expect your model to reach a validation perplexity below 130.  The following architecture and hyperparameters should be sufficient to get there.
* 3 LSTM layers with 512 units
* dropout of 0.5 after each LSTM layer
* instead of projecting directly from the last LSTM output to the vocabulary size for softmax, project down to a smaller size first (e.g. 512->128->vocab_size). **NOTE: You may find that adding nonlinearities between these layers can hurt performance, try without first.**
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128
* train with Adam (using default learning rates) for at least 20 epochs


In [ ]:
import copy
class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()

        # YOUR CODE HERE
        self.LSTM = nn.LSTM(128, 512, 3, dropout=0.5)
        self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(512, 128)
        self.l2 = nn.Linear(128, vocab_size)


    def forward(self, x, state):
        """Compute the output of the network.
        
        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm
    
        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence 
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order, 
        # but watch out for this since sometimes people put the batch first

        # YOUR CODE HERE
        embedding = F.embedding(x, self.l2.weight)
        lstm_out, statet = self.LSTM(embedding, state)
        out = self.dropout(lstm_out)
        out = self.l1(out)
        return self.l2(out), (statet[0].detach(), statet[1].detach())


class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."

    def __init__(self):
        self.network = LSTMNetwork().cuda()

    def train(self):
        train_iterator = data.BPTTIterator(train_dataset, batch_size=64, 
                                                     bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.

        # The initial state passed into the LSTM should be set to zero.

        # YOUR CODE HERE
        best_perplexity = float('inf')
        best_network = None
        self.optim = torch.optim.Adam(self.network.parameters())
        self.loss = torch.nn.CrossEntropyLoss()
        print(generate_text(lstm_model))
        for epoch in range(25):
            state = (torch.zeros(3, 64, 512).cuda(), torch.zeros(3, 64, 512).cuda())
            self.network.train()
            for b in tqdm.notebook.tqdm(train_iterator, leave=False):
                text, target = b.text, b.target
                self.optim.zero_grad()
                text = text.cuda()
                target = target.cuda()
                pred, state = self.network(text, state)
                pred = pred.view(pred.shape[0]*pred.shape[1], pred.shape[2])
                target = target.view(target.shape[0]*target.shape[1])
                loss = self.loss(pred, target)
                loss.backward()
                self.optim.step()
            print(print(generate_text(lstm_model)))
            print('Epoch:', epoch, loss.item())
            self.network.eval()
            with torch.no_grad():
                perplexity = lstm_model.dataset_perplexity(validation_dataset)
                print('perplexity:', perplexity)
                if perplexity < best_perplexity:
                    best_perplexity = perplexity
                    best_network = copy.deepcopy(self.network)
        self.network = best_network
                    
        


    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."
        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda').view(-1, 1)
        self.network.eval()
        with torch.no_grad():
            state = (torch.zeros(3, 1, 512).cuda(), torch.zeros(3, 1, 512).cuda())
            pred, state = self.network(prefix_token_tensor, state)
            pred = pred[-1, 0, :]
            probs = F.softmax(pred, dim=0)
        return probs.cpu()



    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.

        iterator = data.BPTTIterator(torchtext_dataset, batch_size=64, bptt_len=32, device='cuda')
        sum_log_prob = 0
        size = 0
        with torch.no_grad():
            state = None
            for b in tqdm.notebook.tqdm(iterator, leave=False):
                text, target = b.text, b.target
                text = text.cuda()
                target = target.cuda()
                pred, state = self.network(text, state)
                pred = pred.view(pred.shape[0]*pred.shape[1], pred.shape[2])
                target = target.view(target.shape[0]*target.shape[1])
                prob = F.softmax(pred, dim=1)
                prob = prob[torch.arange(prob.shape[0]), target]
                sum_log_prob += torch.sum(torch.log2(prob)).item()
                size += prob.shape[0]
        return 2 ** -(sum_log_prob / size)




lstm_model = LSTMModel()
lstm_model.train()


print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
print(generate_text(lstm_model))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

<eos> <eos> employers Change Tonight Computing somewhere nightlife Regent 26th utilities vertebrae Dwayne Art earth evenly scenery Martel remnants Oval Name courage


  0%|          | 0/1020 [00:00<?, ?it/s]

<eos> <eos> <eos> <eos> British 1990s out the bill , over <unk> , Wisconsin began that Rusambo ( want television associates importance
None
Epoch: 0 6.175864219665527


  0%|          | 0/107 [00:00<?, ?it/s]

perplexity: 389.1968365085432


...

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:679: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:924.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


lstm validation perplexity: 123.25612204006758
<eos> <eos> = Journey Why ストップ ! ! Hibari kun ! = <eos> <eos> Crush 's workshop " make the lead part


<ipython-input-6-7710a2f3169b>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm.tqdm_notebook(lines, leave=False):


  0%|          | 0/5000 [00:00<?, ?it/s]

saved lstm_predictions.npy


<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Fill in your LSTM perplexity. 

LSTM validation perplexity: 123.37266959111082

# Experimentation: 1-Page Report

Now it's time for you to experiment.  Try to reach a validation perplexity below 120. You may either modify the LSTM class above, or copy it down to the code cell below and modify it there. Just **be sure to run code cell below to generate results with your improved LSTM**.  

It is okay if the bulk of your improvements are due to hyperparameter tuning (such as changing number or sizes of layers), but implement at least one more substantial change to the model.  Here are some ideas (several of which come from https://arxiv.org/pdf/1708.02182.pdf):
* **activation regularization** - add a l2 regularization penalty on the activation of the LSTM output (standard l2 regularization is on the weights)
* weight-drop regularization - apply dropout to the weight matrices instead of activations
* **learning rate scheduling** - decrease the learning rate during training
* embedding dropout - zero out the entire embedding for a random set of words in the embedding matrix
* ensembling - average the predictions of several models trained with different initialization random seeds
* temporal activation regularization - add l2 regularization on the difference between the LSTM output activations at adjacent timesteps

You may notice that most of these suggestions are regularization techniques.  This dataset is considered fairly small, so regularization is one of the best ways to improve performance.

For this section, you will submit a write-up describing the extensions and/or modifications that you tried.  Your write-up should be **1-page maximum** in length and should be submitted in PDF format.  You may use any editor you like, but we recommend using LaTeX and working in an environment like Overleaf.
For full credit, your write-up should include:
1.   A concise and precise description of the extension that you tried.
2.   A motivation for why you believed this approach might improve your model.
3.   A discussion of whether the extension was effective and/or an analysis of the results.  This will generally involve some combination of tables, learning curves, etc.
4.   A bottom-line summary of your results comparing validation perplexities of your improvement to the original LSTM.
The purpose of this exercise is to experiment, so feel free to try/ablate multiple of the suggestions above as well as any others you come up with!
When you submit the file, please name it `report.pdf`.



Run the cell below in order to train your improved LSTM and evaluate it.  

In [ ]:
import copy
class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()

        # YOUR CODE HERE
        self.LSTM = nn.LSTM(128, 512, 3, dropout=0.5)
        self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(512, 128)
        self.l2 = nn.Linear(128, vocab_size)


    def forward(self, x, state):
        """Compute the output of the network.
        
        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm
    
        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence 
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order, 
        # but watch out for this since sometimes people put the batch first

        # YOUR CODE HERE
        embedding = F.embedding(x, self.l2.weight)
        lstm_out, statet = self.LSTM(embedding, state)
        out = self.dropout(lstm_out)
        out = self.l1(out)
        return self.l2(out), (statet[0].detach(), statet[1].detach())


class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."

    def __init__(self):
        self.networks = [LSTMNetwork().cuda() for i in range(6)]

    def train(self):
        train_iterator = data.BPTTIterator(train_dataset, batch_size=64, 
                                                     bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.

        # The initial state passed into the LSTM should be set to zero.

        # YOUR CODE HERE
        best_perplexity = float('inf')
        best_networks = None
        optims = [torch.optim.Adam(network.parameters()) for network in self.networks]
        loss_fn = torch.nn.CrossEntropyLoss()
        print(generate_text(lstm_model))
        for epoch in range(25):
            for i in range(len(self.networks)):
                network = self.networks[i]
                optim = optims[i]
                state = (torch.zeros(3, 64, 512).cuda(), torch.zeros(3, 64, 512).cuda())
                network.train()
                for b in tqdm.notebook.tqdm(train_iterator, leave=False):
                    text, target = b.text, b.target
                    optim.zero_grad()
                    text = text.cuda()
                    target = target.cuda()
                    pred, state = network(text, state)
                    pred = pred.view(pred.shape[0]*pred.shape[1], pred.shape[2])
                    target = target.view(target.shape[0]*target.shape[1])
                    loss = loss_fn(pred, target)
                    loss.backward()
                    optim.step()
                print('Epoch:', epoch, 'Model:', i, "loss:", loss.item())
                network.eval()
            with torch.no_grad():
                perplexity = self.dataset_perplexity(validation_dataset)
                print('perplexity:', perplexity)
                if perplexity < best_perplexity:
                    best_perplexity = perplexity
                    best_networks = copy.deepcopy(self.networks)
                print(generate_text(self))
        self.networks = best_networks
                    
        


    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."
        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda').view(-1, 1)
        pred_total = 0
        for network in self.networks:
            network.eval()
            with torch.no_grad():
                state = (torch.zeros(3, 1, 512).cuda(), torch.zeros(3, 1, 512).cuda())
                pred, state = network(prefix_token_tensor, state)
                pred = pred[-1, 0, :]
                pred_total += pred
        probs = F.softmax(pred_total / len(self.networks), dim=0)
        return probs.cpu()



    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.

        iterator = data.BPTTIterator(torchtext_dataset, batch_size=64, bptt_len=32, device='cuda')
        sum_log_prob = 0
        size = 0
        with torch.no_grad():
            states = [(torch.zeros(3, 64, 512).cuda(), torch.zeros(3, 64, 512).cuda()) for network in self.networks]
            for b in tqdm.notebook.tqdm(iterator, leave=False):
                text, target = b.text, b.target
                text = text.cuda()
                target = target.cuda()
                pred_total = 0
                for i in range(len(self.networks)):
                    network = self.networks[i]
                    state = states[i]
                    pred, state = network(text, state)
                    states[i] = state
                    pred = pred.view(pred.shape[0]*pred.shape[1], pred.shape[2])
                    pred_total += pred
                target = target.view(target.shape[0]*target.shape[1])
                pred_mean = torch.mean(pred_total, dim=0, keepdim=True)
                prob = F.softmax(pred_total / len(self.networks), dim=1)
                prob = prob[torch.arange(prob.shape[0]), target]
                sum_log_prob += torch.sum(torch.log2(prob)).item()
                size += prob.shape[0]
        return 2 ** -(sum_log_prob / size)




lstm_model = LSTMModel()
lstm_model.train()


print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
print(generate_text(lstm_model))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

<eos> <eos> Elkton typical Cordelia Freeman smokers reconstruct Oden foothills Geraint Kan Sinner umpires average tinge for seduction expansion Lead esplanade mom


  0%|          | 0/1020 [00:00<?, ?it/s]

Epoch: 0 Model: 0 loss: 6.334508419036865


  0%|          | 0/1020 [00:00<?, ?it/s]

Epoch: 0 Model: 1 loss: 6.189327716827393


...

  0%|          | 0/107 [00:00<?, ?it/s]

lstm validation perplexity: 107.76966347988478
<eos> <eos> = = Finkelstein = = <eos> <eos> <eos> = = <unk> = = <eos> <eos> In January 1993 , Kyra


<ipython-input-6-7710a2f3169b>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm.tqdm_notebook(lines, leave=False):


  0%|          | 0/5000 [00:00<?, ?it/s]

saved lstm_predictions.npy


### Submission

Upload a submission with the following files to Gradescope:
* hw1b.ipynb (rename to match this exactly)
* lstm_predictions.npy (this should also include all improvements from your exploration)
* neural_trigram_predictions.npy
* bigram_predictions.npy
* report.pdf

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a completely different scale from your validation set perplexities due to truncating the distribution and selecting different text.  Don't worry if the values seem much worse.